# Industrial Robot Updating Control Problem 

"Dataset and Baseline for an Industrial Robot Identification Benchmark" paper
Solving the problem of feed forward torque, black box model. 

We will solve in two different methodologies
- Real Time Update
- FoKL Prediction

## Data Evaluation
For feed forward torque modeling the paper supplies an 18 x n data set where each of the 6 joints' angle, angular velocity, and angular acceleration were tracked, and used to model the feed forward torque required to get to the next state.

Due to the nature of GP's this is not ideal for training a model, as the input space increases by '' for each input and will thus very quickly grow larger as it increases. This means some data manipulation must be performed prior to the fitting. The 18 dimensional input space was reduced using a model of the arm, where the distance of the joints from the axis of rotation squared was used as an assumed 'Moment', and velocity and accelerations. This left a 3xn data set which is much more suited for GP fitting.  

## FoKL Prediction
FoKL is a machine learning technique which is scalable, fast training, and fast evaluating. It does this by carefully selecting terms for removal based on hyperparameters.  

## Real Time Update
The updating model functionality is unable to work within the FoKL framework due to the variable selection of terms. Previously discarded terms would need to be resampled and potentially selected, this forced resampling means that the advantages of variable selection would be lost. 

In [1]:
# Import Relevant Libraries

# For Data Processing
import numpy as np
import pandas as pd
from src.FoKL import FoKLRoutines
from Rotations import Arm

model = FoKLRoutines.FoKL()

modelF = FoKLRoutines.FoKL()

# For Graphing
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "browser"

df = pd.DataFrame()
# Import Robot Arm Data

u_test = pd.read_csv("u_test.csv", header=None).to_numpy()
u_train = pd.read_csv("u_train.csv", header=None).to_numpy()
y_test = pd.read_csv("y_test.csv", header=None).to_numpy()
y_train = pd.read_csv("y_train.csv", header=None).to_numpy()


# Creation of Arm Simulation
In another module, a set of rotational matrices are designed such that the joint angles given in the paper can be transferred into coordinates in 3D space. This simulation is performed by first creating the arm object. The physical lengths of the links are inputted in with the angles from the data to perform the simulation.

In [2]:
# Initialize Arm
l1 = [350, 0, 0]
l2 = [1150, 0, 0]
l3 = [1000,0,0]
l4 = [5000,0,0]

initial_angles_train = np.array(u_train[:,0])
initial_angles_test = np.array(u_test[:,0])
# Create arm object
ArmTrain = Arm(l1,l2,l3,l4,initial_angles_train,u_train)
ArmTest = Arm(l1,l2,l3,l4,initial_angles_test,u_test)
# simulate arm motion, receive inputs
Mtrain,Vtrain,Atrain = ArmTrain.simulate()
Mtest,Vtest,Atest = ArmTest.simulate()

# Now perform input manipulation
inputsTrain = [Mtrain,Vtrain,Atrain]
inputsTest = [Mtest,Vtest,Atest]
dataTrain = y_train[0,:]
dataTest = y_test[0,:]

# Model Setup
Next the hyperparameters are set up for the BSS_Anova model. 


In [5]:
# Hyperparameter set up

# sigsqd0
model.sigsqd0 = 0.009

# a (inverse gamma distribution shape factor for data observation error variance)
model.a = 9
modelF.a = 9


# b (inverse gamma distribution scale factor for data observation error variance)
model.b = 0.01
modelF.b = 0.01

# atau (inverse gamma distribution shape factor for beta prior variance)
model.atau = 3
modelF.atau = 3

# btau (inverse gamma distribution scale factor for beta prior variance)
model.btau = 4000
modelF.btau = 4000

# tolerance (number of times needed to run Gibbs Sampling with a higher BIC than previous lowest (best) value)
model.tolerance = 3
modelF.tolerance = 3

# Allows for exclusion of term combinations (in this case none)
model.relats_in = []
modelF.relats_in = []

# draws from the posterior for each tested model
model.draws = 1000
modelF.draws = 1000

model.gimmie = False
modelF.gimmie = False
model.aic = False
modelF.aic = False

# New input hypers
model.update = True
model.built = False
model.burnin = 0

NameError: name 'modelF' is not defined

# Model Setup Cont.
Due to the nature of an updating dating set, min and max criteria must be defined prior to normalization, or new inputs that would fall outside the extrema would result in a changing normalization and invalidate the model. Therefore, a manipulation of normalization and then a manual definition of inputs and data is performed prior to fitting.

In [4]:
# Automatically normalize and format:
model.cleanFun(inputsTrain,dataTrain)

x = model.inputs
y = model.data

modelF.inputs = x
modelF.data = y

/Users/derekslack/PycharmProjects/FoKL-GPy-main-8/src/FoKL/FoKLRoutines.py:297: UserWarning:

'inputs' was transposed. Ignore if more datapoints than input variables, else set 'AutoTranspose=False' to disable.



# FoKL Fit Check
Next we can perform a FoKL fit across the entirity of the data

 

In [ ]:
modelF.fit()
modelF.coverage3()